<img src="images/fail-case.gif" width=40% align="right">

Foundations
===========

*Dask is a way to represent computations as dictionaries, and then analyze and execute them.*

Dask supports parallel computing.  Internally it executes graphs of tasks with data dependencies.  In this section we talk about what these graphs look like and how to construct them.  We finish with exercises manually building graphs that use basic Pandas functionality.  This is straightforward but somewhat tedious.  We'll automate it in future sections.

You can safely skip this section if you don't care about how dask works internally.

**Related Documentation**

*  [Dask graph specification](http://dask.pydata.org/en/latest/spec.html)
*  [Discussion on custom graphs](http://dask.pydata.org/en/latest/custom-graphs.html)

## Normal Programming

Normally we write functions and then use those function in linear code.  The Python interpreter executes this code from the top down.

### Make functions

In [ ]:
def inc(x):
    return x + 1

def add(x, y):
    return x + y

### Call functions in code

In [ ]:
a = 1
b = inc(a)

x = 10
y = inc(x)

z = add(b, y)
z

Even though some of this work could have happened in parallel Python went ahead and executed one line after the other.

If we want to execute code in parallel then we need to stop Python from taking control.

### Computation as a data structure

Instead of writing normal code we store the steps of the computation above as a Python dictionary where each function call becomes a Python tuple.

This is going to look a little strange but we'll have the entire computation stored in a Python data structure that we can manipulate with *other* Python code.

In [ ]:
dsk = {'a': 1, 
       'b': (inc, 'a'),
       
       'x': 10,
       'y': (inc, 'x'),
       
       'z': (add, 'b', 'y')}

In [ ]:
type(dsk)

We call a dictionary that looks like this a *dask graph*.  A dask graph is just a dictionary.

### Related Approaches to Delayed Evaluation

Representing Python functions as tuples containing function names and arguments may seem strange, but in reality you are already familiar with the style.


In [ ]:
# Sometimes we defer compuations with a lambda

x = 15
y = 30
z = lambda: x + y

# z delays the execution of x + y until we call z()
# This is very similar to (add, 'x', 'y')

# Sometimes we defer computations with strings

z = "x + y"
print eval(z)

# The variable 'z' stores a string that is a valid Python statement
# We call eval to fully evaluate `z' and obtain the answer.

# Sometimes we use functools.partial
import functools
z = functools.partial(add, x, y)

# Dask delays evaluation with tuples
z = (add, x, y)

### TODO: further define dask graphs

### Execute dask graphs

The dask library contains functions to execute these dictionaries in parallel with multiple threads or multiple processes.

In [ ]:
from dask.threaded import get
get(dsk, 'z')  # Execute in multiple threads

In [ ]:
from dask.multiprocessing import get
get(dsk, 'z')  # Execute in multiple processes

So as long as you're willing to write code in this funny way with dictionaries, dask will run your separate functions in parallel.

### Analyze and Visualize Graphs

Because our computation is just a dictionary we can write arbitrary functions to do a variety of useful analyses on these dictionaries.  A simple yet common operation is just to visualize the computation as a visual graph. 

In [ ]:
# Requires that you have pydot and graphviz installed
# This isn't a problem if this doesn't work for you
from dask.dot import dot_graph
dot_graph(dsk)

That's it
----------

The rest of this tutorial is fancy ways to construct and executing dask graphs.  We won't make any more by hand after this notebook.  If you'd like to learn more, read the [dask graph spec](http://dask.pydata.org/en/latest/spec.html).

Exercise - `read_csv`
------------------------

As an exercise we'll parallelize some basic Pandas code by rewriting it as a dask graph.  This will be a little tedious but should give us speedups right away.  In future sections we'll learn how dask submodules like `dask.dataframe` automate this work for us.

There are three CSV files in your `data` directory.   We count how many rows are in all of these csv files total.  In normal Python we solve this problem in the following way.

In [ ]:
import pandas as pd

import os
filenames = [os.path.join('data', 'accounts.%d.csv' % i) for i in [0, 1, 2]]
filenames

In [ ]:
pd.read_csv(filenames[0], nrows=5)  # a sample of the first file

In [ ]:
%%time 

a = pd.read_csv(filenames[0])
b = pd.read_csv(filenames[1])
c = pd.read_csv(filenames[2])

na = len(a)
nb = len(b)
nc = len(c)

total = sum([na, nb, nc])
total

### Construct a dask graph/dictionary for this computation

Just as we turned code that looks like 

```python
y = f(x)
```

into dictionaries like 

```python
{'y': (f, 'x')}
```

We can transform the above calls to `pd.read_csv`, `len`, and `sum` into a dictionary of tuples

In [ ]:
dsk = {'a': (pd.read_csv, filenames[0]),
       'b': ...,
       ...
       'total': ...}

In [ ]:
# Solution
%load solutions/Foundations-01.py

### Execute your dask graph

We execute dask graphs with the `get` functions.  There is a get function for both multi-threading and multi-processing.  Get takes two arguments

    get(dsk, output_key)

Run the following cells and see how each get function performs.  Why is there a difference?

In [ ]:
from dask.threaded import get
%time get(dsk, 'total')

In [ ]:
from dask.multiprocessing import get
%time get(dsk, 'total')

### Exercise: Sum of amounts

As a slightly more complex example we'll compute the sum of the amounts column in each CSV file and then add up these sums to get the total amount over all CSV files.

To make the graph construction slightly more challenging we'll use Python for loops rather than write out every entry by hand.

In normal sequential code we might execute the following:

In [ ]:
sums = list()
for fn in filenames:
    df = pd.read_csv(fn)
    sums.append(df.amount.sum())
total = sum(sums)
total

Now create the same computation as a dask graph.

We suggest building and using a small function to compute the sum of the amount of a dataframe and using this function in your dask graph.

In [ ]:
def amount_sum(df):
    return df.amount.sum()

In [ ]:
dsk = dict()

for fn in filenames:
    dsk[...] = ...
...

In [ ]:
# Solution
%load solutions/Foundations-02.py

In [ ]:
get(dsk, 'total')

Conclusion
------------

We've learned about how dask graph represent computations and how we can execute these computations with dask schedulers / get functions.  We've made a few of these dictionaries by hand.  It's straightforward but perhaps tiresome.

In the next sections we'll play with systems that generate these dictionaries for us.